# Transfer Learning in Keras

In this notebook, we'll cover how to load a pre-trained model (in this case, VGGNet19) and finetune it for a new task: detecting hot dogs.

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#### Load dependencies

In [2]:
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


#### Load the pre-trained VGG19 model

In [3]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

#### Freeze all the layers in the base VGGNet19 model

In [4]:
for layer in vgg19.layers:
    layer.trainable = False

#### Add custom classification layers

In [5]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

#### Compile the model for training

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
output_dir = 'model_output/transfer_VGG'

In [8]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+
                                 "/weights.{epoch:02d}.hdf5")

In [9]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Prepare the data for training

The dataset is available for download [here](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/home). You should download the zipfile and extract the contents into a folder called `hot-dog-not-hot-dog` in the `notebooks` directory.

In [10]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [11]:
# Define the batch size:
batch_size=32

In [12]:
# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


#### Train!

In [13]:
model.fit_generator(train_generator, steps_per_epoch=15, 
                    epochs=16, validation_data=valid_generator, 
                    validation_steps=15, callbacks=[modelcheckpoint])

Epoch 1/16
15/15 [==============================] - 27s - loss: 0.9057 - acc: 0.5708 - val_loss: 0.9379 - val_acc: 0.5771
Epoch 2/16
15/15 [==============================] - 5s - loss: 0.7060 - acc: 0.7055 - val_loss: 0.5858 - val_acc: 0.7115
Epoch 3/16
15/15 [==============================] - 4s - loss: 0.6657 - acc: 0.7076 - val_loss: 0.7819 - val_acc: 0.6708
Epoch 4/16
15/15 [==============================] - 4s - loss: 0.4218 - acc: 0.8135 - val_loss: 0.4329 - val_acc: 0.7906
Epoch 5/16
15/15 [==============================] - 4s - loss: 0.3648 - acc: 0.8571 - val_loss: 0.4473 - val_acc: 0.7927
Epoch 6/16
15/15 [==============================] - 4s - loss: 0.3965 - acc: 0.8159 - val_loss: 0.4762 - val_acc: 0.7799
Epoch 7/16
15/15 [==============================] - 4s - loss: 0.3415 - acc: 0.8472 - val_loss: 0.5320 - val_acc: 0.7500
Epoch 8/16
15/15 [==============================] - 4s - loss: 0.2863 - acc: 0.8738 - val_loss: 0.4313 - val_acc: 0.7927
Epoch 9/16
15/15 [=============